# Modelo de Regressão Logística 
###### Modelo Individual Giorgia

## Introdução

A seguir, apresento a modelagem de um **modelo de Regressão Logística** aplicado ao projeto da **Chilli Beans**, com o objetivo de prever o desempenho de suas lojas.  
Essa técnica foi escolhida por sua capacidade de **classificação binária**, permitindo estimar a probabilidade de uma unidade apresentar **Bom Desempenho** ou **Baixo Desempenho** a partir de variáveis explicativas.  

Além de avaliar o desempenho atual, o modelo também funciona como uma **ferramenta de apoio estratégico**, capaz de indicar os **melhores locais para a abertura de novas lojas pelo Brasil**, com base nos padrões identificados nas variáveis de maior impacto.  

O processo de modelagem foi estruturado em etapas: **preparação e tratamento dos dados**, **divisão em treino e teste**, **normalização**, **codificação de variáveis categóricas**, **ajuste de hiperparâmetros** e **interpretação dos coeficientes**. Essa abordagem garante não apenas a robustez estatística, mas também a geração de **insights práticos** que podem orientar tanto a **expansão da rede** quanto a **otimização de canais e produtos já existentes**.  


In [ ]:
# 1 - Importação das bibliotecas
import os
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, confusion_matrix, classification_report, roc_curve
)
import matplotlib.pyplot as plt
import seaborn as sns
import joblib

# Exibir mais colunas no pandas
pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 200)


### Criação da Variável Alvo e Preparação do Dataset
- No primeiro passo, foram carregados dois datasets: o df_limpo, e o df_codificado. 
- Em seguida, a receita total de cada loja foi agregada, e a mediana dessas receitas foi utilizada como limiar para definir a variável alvo Bom_Desempenho. 

- Lojas com receita igual ou superior à mediana foram classificadas como de bom desempenho (1), enquanto as demais foram classificadas como de baixo desempenho (0). 

Essa variável foi então incorporada ao dataset codificado, garantindo que cada registro possuísse a informação do desempenho da respectiva loja. O objetivo desse processamento é transformar o problema em uma tarefa de classificação binária, permitindo que modelos preditivos aprendam padrões que diferenciam lojas de alto e baixo desempenho.

In [ ]:
# 2. Carregar dataset limpo
df_limpo = pd.read_csv("../../../database/dataset gerado/dataset_limpo.csv")

# Agregar receita total por loja
desempenho_loja = df_limpo.groupby("ID_Loja")["Total_Preco_Liquido"].sum().reset_index()

# Definir threshold (mediana da receita total)
threshold = desempenho_loja["Total_Preco_Liquido"].median()

# Criar variável alvo
desempenho_loja["Bom_Desempenho"] = (desempenho_loja["Total_Preco_Liquido"] >= threshold).astype(int)

# Agora juntamos essa variável ao dataset codificado
df_cod = pd.read_csv("../../../database/dataset gerado/dataset_codificado.csv")

df_cod = df_cod.merge(desempenho_loja[["ID_Loja", "Bom_Desempenho"]], on="ID_Loja", how="left")

print(df_cod[["ID_Loja", "Bom_Desempenho"]].head())


In [ ]:
# 3. Carregar o dataset

df_limpo = pd.read_csv("../../../database/dataset gerado/dataset_limpo.csv")
df_codificado = pd.read_csv("../../../database/dataset gerado/dataset_codificado.csv")

print("Dataset limpo:", df_limpo.shape)
print("Colunas disponíveis:", df_limpo.columns.tolist())

print("Dataset codificado:", df_codificado.shape)
print("Colunas disponíveis:", df_codificado.columns.tolist())

display(df_limpo.head())


_O dataset final, pronto para a modelagem, consolidou-se com 31.406 observações e 27 features. Uma característica crucial identificada foi o forte desbalanceamento de classes, com aproximadamente 83% das lojas pertencendo à classe de "Bom Desempenho". Este é um fator que demanda atenção especial na etapa de treinamento, a fim de garantir que o modelo aprenda a identificar corretamente ambas as classes, e não apenas a majoritária._

### Preparação dos Dados para Modelagem
Neste bloco, finalizamos a preparação dos dados: selecionamos apenas as features importantes para compor o conjunto X e isolamos nossa variável-alvo em y. 
Com isso, traduzimos nosso problema de negócio em um formato limpo e estruturado, pronto para ser resolvido por um modelo de machine learning.

In [ ]:

# 4. Lista de colunas a serem removidas
to_drop = [
    "ID_Loja", "ID_Cliente", "ID_Produto",
    "Dim_Cliente.Data_Nascimento", "Dim_Lojas.Nome_Emp", "Dim_Lojas.Bairro_Emp",
    "Dim_Lojas.Cidade_Emp", "Dim_Lojas.Estado_Emp", "Dim_Lojas.Regiao",
    "Dim_Produtos.Grupo_Produto", "Dim_Produtos.GRUPO_CHILLI"
]
to_drop_existing = [c for c in to_drop if c in df_cod.columns]

# Separação de features (X) e alvo (y) a partir de df_cod
X = df_cod.drop(columns=to_drop_existing + ["Bom_Desempenho"], errors="ignore")

# Preencher valores ausentes
y = df_cod["Bom_Desempenho"].fillna(0).astype(int)

print("Shape X:", X.shape, "Shape y:", y.shape)
print("Algumas features (head):")
display(X.head(2))


### Tratamento de Valores Ausentes e Conversão de Tipos
Neste bloco, foi feita uma verificação de valores ausentes em todas as features (X).
Para garantir consistência dos dados:
- Colunas numéricas tiveram valores faltantes preenchidos com a mediana.
- Colunas categóricas codificadas ou one-hot foram preenchidas com 0.

Por fim, todas as colunas foram convertidas para tipos numéricos, preenchendo eventuais NaN remanescentes com 0.

In [ ]:
# 5 - Tratar valores ausentes e converter tipos
# Ver número de missing por coluna
miss = X.isna().sum()
miss = miss[miss > 0].sort_values(ascending=False)
print("Colunas com missing:\n", miss)

# - numéricos: preencher com mediana
# - categóricos codificados/one-hot: preencher com 0
num_cols = X.select_dtypes(include=[np.number]).columns.tolist()
cat_cols = [c for c in X.columns if c not in num_cols]

for c in num_cols:
    med = X[c].median()
    X[c].fillna(med, inplace=True)
for c in cat_cols:
    X[c].fillna(0, inplace=True)

# Garantir que todas as colunas são numéricas
X = X.apply(pd.to_numeric, errors="coerce").fillna(0)



_**Conclusão**: O dataset X ficou totalmente limpo, sem valores ausentes, garantindo que o modelo de regressão logística possa ser treinado sem problemas de inconsistência de dados._

### Divisão Estratificada dos Dados em Treino e Teste
Neste passo, dividimos nosso dataset em dois conjuntos: um para treinar o modelo (80% dos dados) e outro, completamente separado, para testar sua performance (20%).

A parte mais importante foi usar o parâmetro stratify=y.
Como 83% das nossas lojas são de "Bom Desempenho", essa opção garante que essa mesma proporção seja mantida tanto no treino quanto no teste. 

In [ ]:
# 6 - Split treino/teste
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42, stratify=y
)
print("Treino:", X_train.shape, "Teste:", X_test.shape)
print("Proporção target (treino):", y_train.mean(), " (teste):", y_test.mean())


O output confirma que isso foi feito com sucesso, nos dando a certeza de que a avaliação final do modelo será justa e representativa da realidade dos nossos dados.

### Pipeline, GridSearchCV e Treinamento do Modelo
Neste bloco, foi criado um pipeline que primeiro aplica StandardScaler para normalizar as features e depois treina um modelo de Regressão Logística com penalização balanceada para lidar com o desbalanceamento de classes.

Em seguida, utilizou-se o GridSearchCV com validação cruzada estratificada para encontrar os melhores hiperparâmetros (C e penalty) otimizando a métrica ROC AUC.

In [ ]:
# 7 - Pipeline e GridSearchCV
pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("clf", LogisticRegression(max_iter=2000, solver="liblinear", class_weight="balanced"))
])

param_grid = {
    "clf__C": [0.01, 0.1, 1, 5, 10],
    "clf__penalty": ["l1", "l2"]
}

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

grid = GridSearchCV(
    pipeline, param_grid,
    scoring="roc_auc", cv=cv, n_jobs=-1, verbose=1
)

grid.fit(X_train, y_train)

print("Melhor score CV (roc_auc):", grid.best_score_)
print("Melhos params:", grid.best_params_)
best_model = grid.best_estimator_


_**Análise:** O modelo encontrou a melhor combinação de parâmetros (C=1 e penalty='l1') com um ROC AUC médio de 0,727, indicando uma boa capacidade do modelo em diferenciar lojas de bom e baixo desempenho._

### Avaliação do Modelo
Neste bloco, o modelo treinado foi avaliado usando o conjunto de teste. Foram calculadas métricas de classificação como accuracy, precision, recall, F1 e ROC AUC, além de gerar o classification report e a matriz de confusão.

- ROC: É um gráfico que mostra o quão bom o modelo é em separar as lojas de "Bom Desempenho" das de "Mau Desempenho".

- AUC: É a nota que o modelo tira nesse gráfico, de 0.5 a 1.0. Quanto mais perto de 1.0, melhor ele é em acertar a classificação.

In [ ]:
# 8 - Avaliação
y_pred = best_model.predict(X_test)
y_proba = best_model.predict_proba(X_test)[:,1]

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))
print("F1:", f1_score(y_test, y_pred))
print("ROC AUC:", roc_auc_score(y_test, y_proba))
print("\nClassification report:\n", classification_report(y_test, y_pred))



In [ ]:
# Matriz de confusão
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(5,4))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=["Ruim","Bom"], yticklabels=["Ruim","Bom"])
plt.xlabel("Previsto")
plt.ylabel("Real")
plt.title("Matriz de Confusão")
plt.show()

In [ ]:

# Curva ROC
fpr, tpr, th = roc_curve(y_test, y_proba)
plt.figure(figsize=(6,5))
plt.plot(fpr, tpr, label=f"AUC = {roc_auc_score(y_test, y_proba):.3f}")
plt.plot([0,1],[0,1],"--", linewidth=0.6)
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve")
plt.legend()
plt.show()


#### Conclusão do output:
O modelo apresenta accuracy de 66%, com alta precisão para a classe de bom desempenho (0,91), mas menor para a classe de baixo desempenho (0,28).
- O recall indica que 66% das lojas de bom desempenho foram corretamente identificadas.
- O ROC AUC de 0,72 confirma uma boa capacidade discriminativa do modelo.
- A matriz de confusão mostra que a maioria dos erros ocorre na classe minoritária (lojas de baixo desempenho), evidenciando o desbalanceamento das classes.


### Interpretação dos coeficientes:
Os coeficientes do modelo indicam o impacto de cada feature na probabilidade de uma loja ser de bom desempenho. Valores positivos aumentam a chance, enquanto valores negativos reduzem. Por exemplo, Total_Preco_Liquido tem forte efeito positivo, mostrando que lojas com maior faturamento tendem a performar melhor. Alguns canais de venda específicos aparecem como negativos, sugerindo que eles podem estar associados a menor desempenho. Essa análise ajuda a identificar quais características das lojas mais influenciam o sucesso.

In [ ]:
# 9 - Interpretacao dos coeficientes
# Recuperar coeficientes do classificador (após scaler os coef correspondem às features)
clf = best_model.named_steps["clf"]
coefs = clf.coef_[0]
features = X.columns

coef_df = pd.DataFrame({
    "feature": features,
    "coef": coefs,
    "odds_ratio": np.exp(coefs)
}).sort_values(by="odds_ratio", ascending=False)

# Top positivos (aumentam chance de Bom_Desempenho)
print("Top 15 que aumentam odds (odds_ratio > 1):")
display(coef_df.head(15))

# Top negativos (reduzem chance de Bom_Desempenho)
print("Top 15 que reduzem odds (odds_ratio < 1):")
display(coef_df.tail(15).sort_values("odds_ratio"))


### Predição de novas localizações:
A função desenvolvida permite estimar o desempenho esperado de novas lojas, retornando a probabilidade de bom desempenho e uma classificação binária. 

In [ ]:
# 10 - Função para predizer novas localizações
def prever_novas_localizacoes(df_novas, model=best_model, features=X.columns):
    """
    Recebe um DataFrame com as mesmas colunas/features que X (codificadas),
    retorna probabilidade de Bom_Desempenho e previsao binaria.
    """
    # garantir colunas na ordem correta, criar colunas faltantes com 0
    df_copy = df_novas.copy()
    for c in features:
        if c not in df_copy.columns:
            df_copy[c] = 0
    df_copy = df_copy[features]
    probs = model.predict_proba(df_copy)[:,1]
    preds = (probs >= 0.5).astype(int)
    return pd.DataFrame({"prob_bom_desempenho": probs, "pred_bom_desempenho": preds})

# Exemplo de uso com amostra do próprio X_test
amostra = X_test.sample(5, random_state=42)
resultado = prever_novas_localizacoes(amostra)
display(amostra.head())
display(resultado)


#### Predição em Cenários de Novas Localizações
Após o treinamento e validação do modelo supervisionado, a próxima etapa foi aplicá-lo em um novo conjunto de dados que não pertence ao dataset original. Esses dados representam cenários de possíveis novas localizações e foram previamente gerados a partir do modelo não supervisionado, responsável por agrupar e simplificar as variáveis relevantes.
A ideia dessa etapa é simular o uso real do modelo: em vez de apenas avaliar seu desempenho em dados históricos, passamos a aplicá-lo em situações inéditas, obtendo a probabilidade de sucesso para cada nova localização. Isso nos permite comparar locais entre si e priorizar aqueles com maior potencial.

In [ ]:
# 1. Carregar dataset das novas localizações
df_novas = pd.read_csv("../../../database/dataset gerado/ranking_grupo_chilli_por_local_simplificado.csv")
# 2. Dropar colunas que não são features (se existirem)
df_novas = df_novas.drop(columns=to_drop_existing, errors="ignore")

# 3. Prever desempenho
resultado_novas = prever_novas_localizacoes(df_novas, model=best_model, features=X.columns)

# 4. Exibir resultados
display(resultado_novas.head(10))


#### Conclusão dos Resultados em Novas Localizações
Os resultados obtidos indicam que, para as novas localizações simuladas a partir do modelo não supervisionado, o classificador atribuiu uma probabilidade extremamente elevada de bom desempenho (próxima de 1 em todos os casos). Isso significa que, segundo os padrões aprendidos pelo modelo supervisionado, esses cenários apresentam características muito semelhantes às unidades já existentes que tiveram performance positiva no histórico.

Do ponto de vista da análise exploratória regional, esse achado reforça dois pontos:
Coerência entre os modelos, os agrupamentos gerados pela análise não supervisionada selecionaram pontos com perfil consistente de sucesso, o que valida a complementaridade das duas abordagens.

- Potencial de expansão homogêneo: a uniformidade nas probabilidades sugere que, para a amostra testada, não há distinções marcantes entre as regiões: todas são vistas pelo modelo como oportunidades favoráveis.

Esse resultado não implica necessariamente que todas as novas localizações terão sucesso garantido na prática, mas mostra que, com base nas variáveis disponíveis, o modelo não identificou sinais de risco relevante em nenhum dos cenários simulados.


## Análise Exploratória Regional 

Para complementar os resultados do modelo, foi realizada uma análise exploratória regional. O objetivo foi entender se padrões de desempenho das lojas variam por região e se as diferenças observadas refletem características reais dos dados, como concentração de lojas ou receita média, em vez de serem apenas um efeito do modelo. Essa abordagem ajuda a interpretar os insights de forma mais contextualizada e a validar que certas regiões apresentam, de fato, maior probabilidade de lojas de bom desempenho.

In [ ]:
# 1. Criar um mapeamento de ID_Loja para Regiao a partir do df_limpo
#    Usamos drop_duplicates para ter certeza que temos uma linha por loja
loja_regiao_map = df_limpo[['ID_Loja', 'Dim_Lojas.REGIAO_CHILLI']].drop_duplicates()

In [ ]:
# 2. Juntar essa informação de região com nosso dataframe de desempenho
#    desempenho_loja já contém ID_Loja, Total_Preco_Liquido (agregado), e Bom_Desempenho
analise_regional_df = pd.merge(desempenho_loja, loja_regiao_map, on='ID_Loja')

In [ ]:
# 3. Agrupar por região e calcular as métricas
sumario_regional = analise_regional_df.groupby('Dim_Lojas.REGIAO_CHILLI').agg(
    N_Lojas=('ID_Loja', 'nunique'),
    Receita_Media_por_Loja=('Total_Preco_Liquido', 'mean'),
    Proporcao_Bom_Desempenho=('Bom_Desempenho', 'mean')
).sort_values(by='N_Lojas', ascending=False)

In [ ]:
# 4. Formatar para melhor visualização
sumario_regional['Proporcao_Bom_Desempenho'] = (sumario_regional['Proporcao_Bom_Desempenho'] * 100).map('{:.2f}%'.format)
sumario_regional['Receita_Media_por_Loja'] = sumario_regional['Receita_Media_por_Loja'].map('R$ {:,.2f}'.format)

print("Análise de Desempenho por Região:")
display(sumario_regional)


### Conclusão Geral e Estratégica

O modelo de regressão logística desenvolvido demonstrou boa capacidade de classificação das lojas da Chilli Beans em Bom Desempenho e Baixo Desempenho, alcançando ROC AUC de 0,72 e acurácia de 66%, mesmo diante do forte desbalanceamento de classes. 

A análise dos coeficientes mostrou que variáveis como receita total, estado, região da loja e canais de venda têm impacto significativo na performance, oferecendo insights diretos sobre os fatores que impulsionam ou limitam o sucesso das unidades.

A análise regional revelou padrões claros: São Paulo concentra o maior número de lojas e apresenta a maior receita média, enquanto Nordeste e Sudeste mostram oportunidades de melhoria. 
Sul e Norte, apesar de possuírem menos lojas, apresentam proporções relativamente altas de bom desempenho, indicando que características locais podem superar a simples quantidade de unidades. 

Esses resultados permitem à empresa priorizar regiões estratégicas, ajustar mix de produtos e canais de venda, e planejar novas aberturas com base em probabilidade de sucesso, minimizando riscos.

Em termos de negócios, o modelo transforma dados históricos em decisões acionáveis, conectando análise quantitativa à performance real das lojas. Ele fornece uma ferramenta prática para otimizar operações, direcionar investimentos e apoiar decisões estratégicas de expansão, mostrando como a inteligência de dados pode gerar vantagem competitiva sustentável para a rede.



---

A seguir, apresentamos gráficos que permitem observar de forma visual o desempenho do modelo, a distribuição das probabilidades previstas, o impacto das principais variáveis e as diferenças regionais. Essas representações facilitam a interpretação dos resultados e ajudam a conectar a análise estatística a insights práticos para decisões estratégicas.

### Distribuição das Probabilidades Previstas

In [ ]:
plt.figure(figsize=(8,5))
sns.histplot(y_proba, bins=30, kde=True, color="skyblue")
plt.title("Distribuição das Probabilidades Previstas de Bom Desempenho")
plt.xlabel("Probabilidade de Bom Desempenho")
plt.ylabel("Quantidade de Lojas")
plt.show()

### Importância das Features

In [ ]:
# Top 10 positivos e negativos
top_features = pd.concat([coef_df.head(10), coef_df.tail(10)])
plt.figure(figsize=(10,6))
sns.barplot(x="odds_ratio", y="feature", data=top_features, palette="coolwarm")
plt.axvline(1, color="black", linestyle="--")
plt.title("Impacto das Features na Probabilidade de Bom Desempenho (Odds Ratio)")
plt.xlabel("Odds Ratio")
plt.ylabel("Feature")
plt.show()

### Proporção de Bom Desempenho por Região

In [ ]:
# Converter porcentagem de string para float para plot
sumario_regional_plot = sumario_regional.copy()
sumario_regional_plot['Proporcao_Bom_Desempenho'] = sumario_regional_plot['Proporcao_Bom_Desempenho'].str.replace('%','').astype(float)

plt.figure(figsize=(8,5))
sns.barplot(x=sumario_regional_plot.index, y='Proporcao_Bom_Desempenho', data=sumario_regional_plot, palette="viridis")
plt.title("Proporção de Lojas de Bom Desempenho por Região")
plt.ylabel("Proporção (%)")
plt.xlabel("Região")
plt.xticks(rotation=45)
plt.show()


### Matriz de Confusão com Porcentagem

In [ ]:
cm = confusion_matrix(y_test, y_pred)
cm_percent = cm / cm.sum(axis=1)[:, None] * 100

plt.figure(figsize=(6,5))
sns.heatmap(cm_percent, annot=True, fmt=".1f", cmap="Blues", xticklabels=["Ruim","Bom"], yticklabels=["Ruim","Bom"])
plt.xlabel("Previsto")
plt.ylabel("Real")
plt.title("Matriz de Confusão (%)")
plt.show()


### Boxplot de Receita por Desempenho

In [ ]:
plt.figure(figsize=(8,5))
sns.boxplot(x="Bom_Desempenho", y="Total_Preco_Liquido", data=desempenho_loja, palette="Set2")
plt.xticks([0,1], ["Baixo Desempenho","Bom Desempenho"])
plt.title("Distribuição da Receita Total por Classe de Desempenho")
plt.ylabel("Receita Total")
plt.show()
